In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
data = pd.read_csv('../input/sms-spam-collection-dataset/spam.csv',encoding='latin1')

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
sns.heatmap(data=data.isna(),annot=False)
plt.show()

In [ ]:
data.isna().sum()

In [ ]:
data.columns

In [ ]:
data.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'],inplace=True,axis=1)

In [ ]:
data.head()

In [ ]:
email_data = data.copy()

In [ ]:
email_data = email_data[['v1','v2']]

In [ ]:
email_data.rename(columns={'v1':"Category",'v2':"Email"},inplace=True)

In [ ]:
email_data.head()

In [ ]:
sns.countplot(x=email_data['Category'])
print(email_data['Category'].value_counts(normalize=True)*100)
plt.show()

In [ ]:
#preprocessing
email_data['Email'] = email_data['Email'].apply(lambda x:" ".join(x.lower() for x in x.split()))

In [ ]:
from nltk.corpus import stopwords
stop_word = stopwords.words('english')

In [ ]:
email_data['Email'] = email_data['Email'].apply(lambda x:" ".join(x for x in x.split() if x not in stop_word))

In [ ]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [ ]:
email_data['Email'] = email_data['Email'].apply(lambda x:" ".join([ps.stem(x) for x in x.split()]))

In [ ]:
from nltk.stem.wordnet import WordNetLemmatizer
lem = WordNetLemmatizer()

In [ ]:
email_data['Email'] = email_data['Email'].apply(lambda x:" ".join([lem.lemmatize(x) for x in x.split()]))

In [ ]:
email_data.head()

In [ ]:
#now splitting the data into number of feature
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
X = email_data['Email']
Y = email_data['Category']

In [ ]:
train_x,test_x,train_y,test_y = train_test_split(X,Y,random_state=1,test_size=0.3)

In [ ]:
from sklearn import naive_bayes,preprocessing,linear_model,metrics,svm
#tfidf encoder to generate minimum of 5000 features
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
test_y = encoder.fit_transform(test_y)

In [ ]:
tfidf_vect = TfidfVectorizer(analyzer='word',token_pattern=r'\w{1,}',max_features=5000)

In [ ]:
tfidf_vect.fit(email_data['Email'])

In [ ]:
try1 = tfidf_vect

In [ ]:
try1.fit_transform(email_data['Email'])

In [ ]:
try1.get_feature_names()

In [ ]:
x_train_tfidf = tfidf_vect.transform(train_x)
x_test_tfidf = tfidf_vect.transform(test_x)

In [ ]:
x_train_tfidf.data

In [ ]:
def train_model(model,feature_vector_train,target_vector,feature_vector_test,is_neural_net=False):
    #fit the training data
    model.fit(feature_vector_train,target_vector)
    #predicting the data
    predication = model.predict(feature_vector_test)
    print("Accuray of model  \n",metrics.accuracy_score(test_y,predication))
    print("classification report of model  \n",metrics.classification_report(test_y,predication))
    print("confusion metics report of model  \n",metrics.confusion_matrix(test_y,predication))
    cm = metrics.confusion_matrix(test_y,predication)
    conf_matrix = pd.DataFrame(data=cm,columns=['Predicated:Spam','Predicated:Ham'],index=['Actual:Spam','Actual:Ham'])
    sns.heatmap(data=conf_matrix,annot=True,fmt='d',cmap='YlGnBu')
    plt.show()

In [ ]:
train_model(naive_bayes.MultinomialNB(alpha=0.2),x_train_tfidf,train_y,x_test_tfidf)

In [ ]:
train_model(linear_model.LogisticRegression(),x_train_tfidf,train_y,x_test_tfidf)